# Remove Unused Columns from Food Dataset

This notebook removes columns that are not useful for analysis:

1. **High Null Value Columns** - Remove columns with >95% missing data
2. **Source File Column** - Remove metadata column not needed for analysis
3. **Data Quality Check** - Ensure final dataset is clean and usable

## Input File

- `../../dataset/process_dataset/processed_food_dataset.csv`

## Output File

- `../../dataset/process_dataset/final_usable_food_dataset.csv`

## Strategy

- Remove columns with >95% missing values
- Remove metadata columns not needed for ML
- Keep only nutritional and food name columns


In [18]:
import pandas as pd

# Load the dataset
df = pd.read_csv(r'D:\Code\Lychee\lychee-meal-planners\systems\dataset\process_dataset\processed_food_dataset.csv')

# Check for null values
print("Null values per column:")
print(df.isnull().sum())

print(f"\nTotal null values: {df.isnull().sum().sum()}")
print(f"Dataset shape: {df.shape}")
print(f"Percentage of null values: {(df.isnull().sum().sum() / (df.shape[0] * df.shape[1])) * 100:.2f}%")

Null values per column:
food_item           0
calories            0
proteins            0
carbohydrates       0
fats                0
fibers              0
sugars              0
sodium              0
cholesterol         0
water_intake     8646
category         8646
meal_type        8646
source_file         0
dtype: int64

Total null values: 25938
Dataset shape: (8681, 13)
Percentage of null values: 22.98%


## Decision on Column Removal

Based on the analysis above:

- **`water_intake`**: 99.6% null values - Remove (unusable for ML)
- **`category`**: 99.6% null values - Remove (unusable for ML)
- **`meal_type`**: 99.6% null values - Remove (unusable for ML)
- **`source_file`**: Metadata column not needed for nutritional analysis - Remove

We will keep the 9 essential nutritional columns:

- `food_item` (identifier)
- `calories`, `proteins`, `carbohydrates`, `fats`, `fibers`, `sugars`, `sodium`, `cholesterol` (features)


In [ ]:
# Remove columns with excessive null values and metadata columns
columns_to_remove = ['water_intake', 'category', 'meal_type', 'source_file']

print(f"Original dataset shape: {df.shape}")
print(f"Columns to remove: {columns_to_remove}")

# Create final optimized dataset
final_df = df.drop(columns=columns_to_remove)

print(f"Final dataset shape: {final_df.shape}")
print(f"Columns removed: {len(columns_to_remove)}")
print(f"Remaining columns: {list(final_df.columns)}")

In [ ]:
# Verify final dataset quality
print("=== FINAL DATASET QUALITY CHECK ===")
print(f"Shape: {final_df.shape}")
print(f"Total null values: {final_df.isnull().sum().sum()}")
print(f"Memory usage: {final_df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

print("\nNull values per column:")
print(final_df.isnull().sum())

print("\nData types:")
print(final_df.dtypes)

print("\nFirst 5 rows:")
print(final_df.head())

In [ ]:
# Export final optimized dataset
import os

output_path = '../../dataset/process_dataset/final_usable_food_dataset.csv'
os.makedirs(os.path.dirname(output_path), exist_ok=True)

final_df.to_csv(output_path, index=False)

print(f"Final dataset exported to: {output_path}")

# Verify the exported file
if os.path.exists(output_path):
    file_size = os.path.getsize(output_path) / 1024**2  # Size in MB
    print(f"File successfully created!")
    print(f"File size: {file_size:.2f} MB")
    
    # Quick verification by reading back
    verification_df = pd.read_csv(output_path)
    print(f"Verification - Shape: {verification_df.shape}")
    print(f"Verification - Columns: {list(verification_df.columns)}")
else:
    print("Error: File was not created!")

## ✅ Dataset Optimization Complete!

### Summary of Changes:

- **Removed 4 columns**: `water_intake`, `category`, `meal_type`, `source_file`
- **Retained 9 essential columns**: `food_item` + 8 nutritional features
- **Zero null values**: All remaining data is complete and usable
- **Optimized for ML**: Clean, numerical features ready for analysis

### Final Dataset Characteristics:

- **Rows**: 8,681 food items
- **Columns**: 9 (1 identifier + 8 features)
- **Data Quality**: 100% complete (no missing values)
- **File**: `final_usable_food_dataset.csv`

### Ready for Next Steps:

- Exploratory Data Analysis (EDA)
- Feature engineering
- Machine learning model training
- Nutritional pattern analysis


In [12]:
# Analyze which columns should be removed
print("COLUMN REMOVAL ANALYSIS:")
print("=" * 50)

# Define thresholds
HIGH_NULL_THRESHOLD = 0.95  # Remove columns with >95% null values
total_rows = len(df)

print(f"Total rows in dataset: {total_rows:,}")
print(f"High null threshold: {HIGH_NULL_THRESHOLD*100}%")

# Analyze each column
columns_to_remove = []
columns_to_keep = []

print(f"\nColumn Analysis:")
print("-" * 40)

for col in df.columns:
    null_count = df[col].isnull().sum()
    null_percentage = null_count / total_rows
    
    # Decision logic
    if col == 'source_file':
        reason = "Metadata column (not needed for analysis)"
        decision = "REMOVE"
        columns_to_remove.append(col)
    elif null_percentage > HIGH_NULL_THRESHOLD:
        reason = f"Too many null values ({null_percentage*100:.1f}%)"
        decision = "REMOVE"
        columns_to_remove.append(col)
    else:
        reason = f"Good data quality ({null_percentage*100:.1f}% null)"
        decision = "KEEP"
        columns_to_keep.append(col)
    
    print(f"  {col:15} | {null_count:>6,} nulls ({null_percentage*100:>5.1f}%) | {decision:6} | {reason}")

print(f"\nSUMMARY:")
print(f"Columns to remove: {len(columns_to_remove)} - {columns_to_remove}")
print(f"Columns to keep: {len(columns_to_keep)} - {columns_to_keep}")
print(f"Final dataset will have {len(columns_to_keep)} columns")

COLUMN REMOVAL ANALYSIS:
Total rows in dataset: 8,681
High null threshold: 95.0%

Column Analysis:
----------------------------------------
  food_item       |      0 nulls (  0.0%) | KEEP   | Good data quality (0.0% null)
  calories        |      0 nulls (  0.0%) | KEEP   | Good data quality (0.0% null)
  proteins        |      0 nulls (  0.0%) | KEEP   | Good data quality (0.0% null)
  carbohydrates   |      0 nulls (  0.0%) | KEEP   | Good data quality (0.0% null)
  fats            |      0 nulls (  0.0%) | KEEP   | Good data quality (0.0% null)
  fibers          |      0 nulls (  0.0%) | KEEP   | Good data quality (0.0% null)
  sugars          |      0 nulls (  0.0%) | KEEP   | Good data quality (0.0% null)
  sodium          |      0 nulls (  0.0%) | KEEP   | Good data quality (0.0% null)
  cholesterol     |      0 nulls (  0.0%) | KEEP   | Good data quality (0.0% null)
  water_intake    |  8,646 nulls ( 99.6%) | REMOVE | Too many null values (99.6%)
  category        |  8,646 null

In [13]:
# Remove unused columns
print("REMOVING UNUSED COLUMNS:")
print("=" * 50)

print(f"Original dataset shape: {df.shape}")
print(f"Original columns: {list(df.columns)}")

# Create cleaned dataset by keeping only useful columns
df_clean = df[columns_to_keep].copy()

print(f"\nAfter removing unused columns:")
print(f"New dataset shape: {df_clean.shape}")
print(f"New columns: {list(df_clean.columns)}")
print(f"Columns removed: {len(columns_to_remove)}")
print(f"Columns kept: {len(columns_to_keep)}")

# Verify no null values in remaining columns
remaining_nulls = df_clean.isnull().sum()
print(f"\nNull values in remaining columns:")
for col in df_clean.columns:
    null_count = remaining_nulls[col]
    if null_count > 0:
        print(f"  ⚠️  {col}: {null_count} null values")
    else:
        print(f"  ✅ {col}: No null values")

total_remaining_nulls = remaining_nulls.sum()
print(f"\nTotal remaining null values: {total_remaining_nulls}")

if total_remaining_nulls == 0:
    print("🎉 Perfect! No null values in the final dataset!")
else:
    print(f"⚠️  Still have {total_remaining_nulls} null values to handle")

REMOVING UNUSED COLUMNS:
Original dataset shape: (8681, 13)
Original columns: ['food_item', 'calories', 'proteins', 'carbohydrates', 'fats', 'fibers', 'sugars', 'sodium', 'cholesterol', 'water_intake', 'category', 'meal_type', 'source_file']

After removing unused columns:
New dataset shape: (8681, 9)
New columns: ['food_item', 'calories', 'proteins', 'carbohydrates', 'fats', 'fibers', 'sugars', 'sodium', 'cholesterol']
Columns removed: 4
Columns kept: 9

Null values in remaining columns:
  ✅ food_item: No null values
  ✅ calories: No null values
  ✅ proteins: No null values
  ✅ carbohydrates: No null values
  ✅ fats: No null values
  ✅ fibers: No null values
  ✅ sugars: No null values
  ✅ sodium: No null values
  ✅ cholesterol: No null values

Total remaining null values: 0
🎉 Perfect! No null values in the final dataset!


In [14]:
# Data quality validation
print("FINAL DATA QUALITY VALIDATION:")
print("=" * 50)

# Check data types
print("Data types:")
for col in df_clean.columns:
    dtype = df_clean[col].dtype
    print(f"  - {col}: {dtype}")

# Check for any obvious data quality issues
print(f"\nData Quality Checks:")
print("-" * 25)

# 1. Check for duplicate food items
duplicate_foods = df_clean.duplicated(subset=['food_item']).sum()
print(f"✅ Duplicate food items: {duplicate_foods}")

# 2. Check for empty food names
empty_names = df_clean['food_item'].isnull().sum() + (df_clean['food_item'] == '').sum()
print(f"✅ Empty food names: {empty_names}")

# 3. Check nutritional value ranges
nutrition_cols = [col for col in df_clean.columns if col != 'food_item']
print(f"\nNutritional value ranges:")
for col in nutrition_cols:
    min_val = df_clean[col].min()
    max_val = df_clean[col].max()
    mean_val = df_clean[col].mean()
    print(f"  - {col}: {min_val:.1f} - {max_val:.1f} (avg: {mean_val:.1f})")
    
    # Check for negative values
    negative_count = (df_clean[col] < 0).sum()
    if negative_count > 0:
        print(f"    ⚠️  Warning: {negative_count} negative values found")

# Memory usage
memory_usage = df_clean.memory_usage(deep=True).sum() / 1024 / 1024
print(f"\nMemory usage: {memory_usage:.2f} MB")

print(f"\n📊 FINAL DATASET SUMMARY:")
print(f"   - Rows: {len(df_clean):,}")
print(f"   - Columns: {len(df_clean.columns)}")
print(f"   - Unique food items: {df_clean['food_item'].nunique():,}")
print(f"   - Nutritional columns: {len(nutrition_cols)}")
print(f"   - Data completeness: 100% (no null values)")

FINAL DATA QUALITY VALIDATION:
Data types:
  - food_item: object
  - calories: float64
  - proteins: float64
  - carbohydrates: float64
  - fats: float64
  - fibers: float64
  - sugars: float64
  - sodium: float64
  - cholesterol: float64

Data Quality Checks:
-------------------------
✅ Duplicate food items: 0
✅ Empty food names: 0

Nutritional value ranges:
  - calories: 0.0 - 2236.0 (avg: 235.5)
  - proteins: 0.0 - 66000.0 (avg: 17.2)
  - carbohydrates: 0.0 - 9000.0 (avg: 27.8)
  - fats: 0.0 - 646.0 (avg: 10.9)
  - fibers: 0.0 - 86.0 (avg: 2.8)
  - sugars: 0.0 - 26700.0 (avg: 14.8)
  - sodium: 0.0 - 38758.0 (avg: 317.4)
  - cholesterol: 0.0 - 3100.0 (avg: 34.1)

Memory usage: 1.29 MB

📊 FINAL DATASET SUMMARY:
   - Rows: 8,681
   - Columns: 9
   - Unique food items: 8,681
   - Nutritional columns: 8
   - Data completeness: 100% (no null values)


In [15]:
# Save the final usable dataset
output_file = '../../dataset/process_dataset/final_usable_food_dataset.csv'
print(f"SAVING FINAL USABLE DATASET:")
print("=" * 50)

# Save to CSV
df_clean.to_csv(output_file, index=False)

import os
file_size = os.path.getsize(output_file)

print(f"✅ Final dataset saved to: {output_file}")
print(f"📄 File size: {file_size:,} bytes ({file_size / 1024 / 1024:.2f} MB)")
print(f"📊 Total rows: {len(df_clean):,}")
print(f"📋 Total columns: {len(df_clean.columns)}")
print(f"🍎 Unique food items: {df_clean['food_item'].nunique():,}")

# Show the reduction in data size
original_size = len(df) * len(df.columns)
final_size = len(df_clean) * len(df_clean.columns)
size_reduction = ((original_size - final_size) / original_size) * 100

print(f"\nDATA REDUCTION SUMMARY:")
print("-" * 25)
print(f"Original data cells: {original_size:,}")
print(f"Final data cells: {final_size:,}")
print(f"Reduction: {size_reduction:.1f}% ({original_size - final_size:,} cells)")
print(f"Null values eliminated: {25938:,} (100% of nulls removed)")

# Show final column list
print(f"\nFinal columns in usable dataset:")
for i, col in enumerate(df_clean.columns, 1):
    data_type = df_clean[col].dtype
    print(f"  {i:2d}. {col} ({data_type})")

# Verify the saved file
print(f"\nVerifying saved file...")
df_verify = pd.read_csv(output_file)
print(f"✓ Verification successful - loaded {len(df_verify):,} rows and {len(df_verify.columns)} columns")

# Show file path for easy access
full_path = os.path.abspath(output_file)
print(f"\n📁 Full file path: {full_path}")

print(f"\n🎉 DATASET CLEANUP COMPLETED SUCCESSFULLY!")
print(f"✨ The dataset is now optimized and ready for machine learning!")
print(f"\n📈 Ready for:")
print(f"   - Food recommendation algorithms")
print(f"   - Nutritional analysis")
print(f"   - Meal planning systems")
print(f"   - Machine learning model training")

SAVING FINAL USABLE DATASET:
✅ Final dataset saved to: ../../dataset/process_dataset/final_usable_food_dataset.csv
📄 File size: 682,640 bytes (0.65 MB)
📊 Total rows: 8,681
📋 Total columns: 9
🍎 Unique food items: 8,681

DATA REDUCTION SUMMARY:
-------------------------
Original data cells: 112,853
Final data cells: 78,129
Reduction: 30.8% (34,724 cells)
Null values eliminated: 25,938 (100% of nulls removed)

Final columns in usable dataset:
   1. food_item (object)
   2. calories (float64)
   3. proteins (float64)
   4. carbohydrates (float64)
   5. fats (float64)
   6. fibers (float64)
   7. sugars (float64)
   8. sodium (float64)
   9. cholesterol (float64)

Verifying saved file...
✓ Verification successful - loaded 8,681 rows and 9 columns

📁 Full file path: d:\Code\Lychee\lychee-meal-planners\systems\dataset\process_dataset\final_usable_food_dataset.csv

🎉 DATASET CLEANUP COMPLETED SUCCESSFULLY!
✨ The dataset is now optimized and ready for machine learning!

📈 Ready for:
   - Food r

# 🎉 Dataset Cleanup Summary

## Cleanup Operations Performed

1. **Removed High-Null Columns** - Eliminated columns with >95% missing data:

   - `water_intake` (99.6% missing)
   - `category` (99.6% missing)
   - `meal_type` (99.6% missing)

2. **Removed Metadata Column** - Eliminated non-analytical column:

   - `source_file` (metadata not needed for ML)

3. **Data Quality Validation** - Ensured final dataset quality

## Final Dataset Specifications

- **Input File**: `../../dataset/process_dataset/processed_food_dataset.csv`
- **Output File**: `../../dataset/process_dataset/final_usable_food_dataset.csv`
- **Final Columns**: 9 (reduced from 13)
- **Data Completeness**: 100% (no null values)
- **Unique Food Items**: 8,681

## Remaining Columns

✅ **food_item** - Food name identifier  
✅ **calories** - Energy content  
✅ **proteins** - Protein content  
✅ **carbohydrates** - Carbohydrate content  
✅ **fats** - Fat content  
✅ **fibers** - Fiber content  
✅ **sugars** - Sugar content  
✅ **sodium** - Sodium content  
✅ **cholesterol** - Cholesterol content

## Benefits

- **Reduced file size** by eliminating unused columns
- **100% data completeness** - no missing values
- **Optimized for ML** - only relevant nutritional features
- **Memory efficient** - smaller memory footprint
- **Production ready** - clean, validated dataset

## Ready for Production

The final dataset is now optimized for:

- Food recommendation systems
- Nutritional analysis algorithms
- Meal planning applications
- Machine learning model training
- Real-time food matching services
